# Fórmula de propagación de errores

Consideramos una variable normal bivariada $X = (X_1, X_2) $ y una variable aleatoria univariada $Y$ obtenida a partir de $X$, $Y = X_1 \, \exp(X_2)$. Conocida la media y la matriz de covarianza de $X$ calculamos la media y varianza de $Y$ mediante la fórmula de propagación de errores. Comparamos los resultados con simulaciones.

Media de $X$

In [ ]:
import numpy as np
mean_x1 = 1
mean_x2 = -1
mean_x = np.array([mean_x1, mean_x2])

Matriz de covarianza de $X$

Probamos los casos en que los errores son chicos y grandes.

Caso errores *chicos*

In [ ]:
sigma_x1 = 0.056
sigma_x2 = 0.048

Caso errores *grandes*

In [ ]:
# sigma_x1 = 0.56
# sigma_x2 = 0.48

In [ ]:
correlation =  -0.85

In [ ]:
import danatools
covariance_x = danatools.covariance_matrix_2d(sigma_x1, sigma_x2, correlation)

## Varianza analítica

In [ ]:
def function(x1, x2):
    return x1 * np.exp(x2)

Media de $Y$

In [ ]:
mean_y_ana = function(mean_x1, mean_x2)
mean_y_ana

Varianza de $Y$

In [ ]:
import math
gradient_x1 = math.exp(mean_x2)
gradient_x2 = mean_x1 * math.exp(mean_x2)

In [ ]:
variance_y_ana = gradient_x1**2 * covariance_x[0,0] + gradient_x2**2 * covariance_x[1,1] + 2 * gradient_x1 * gradient_x2 *  covariance_x[0,1]
variance_y_ana

In [ ]:
sigma_y_ana = math.sqrt(variance_y_ana)
sigma_y_ana

## Varianza simulaciones

In [ ]:
from scipy.stats import multivariate_normal
population = 1000000
rng = np.random.default_rng(seed=6870)
data_x = multivariate_normal.rvs(mean_x, covariance_x, size=population, random_state=rng)
data_x

In [ ]:
data_y = function(data_x[:,0], data_x[:,1])
data_y

In [ ]:
mean_y_sim = data_y.mean()
mean_y_sim

In [ ]:
sigma_y_sim = data_y.std(ddof=1)
sigma_y_sim

## Comparación fórmula analítica con simulaciones

In [ ]:
delta_mean = (mean_y_ana / mean_y_sim - 1) 
print(f"Mean: {delta_mean*100:.2f}%")

In [ ]:
delta_sigma = (sigma_y_ana / sigma_y_sim - 1) 
print(f"Standard deviation: {delta_sigma*100:.2f}%")

## Plot de la PDF conjunta de $X$

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
fig, ax = plt.subplots()
ax.set_xlabel("$X_1$")
ax.set_ylabel("$X_2$")

height_limits = [mean_x1-3*sigma_x1, mean_x1+3*sigma_x1]
weight_limits = [mean_x2-3*sigma_x2, mean_x2+3*sigma_x2]
counts, xedges, yedges, im = ax.hist2d(data_x[:,0], data_x[:,1], range=[height_limits, weight_limits], bins=100, density=True, cmap=cm.viridis)
clb = plt.colorbar(im)

ax.plot(*danatools.get_ellipse(mean_x, covariance_x, nsigma=1), color='tab:red', ls='--')
ax.plot(*danatools.get_ellipse(mean_x, covariance_x, nsigma=2), color='tab:red', ls='--')

## Plot de la PDF de $Y$

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("$Y$")
ax.set_ylabel(" Probability density")

limits = [mean_y_ana-3*sigma_y_ana, mean_y_ana+3*sigma_y_ana] 
ax.hist(data_y, range=limits, bins=100, density=True, label="Simulation")

from scipy.stats import norm
x = np.linspace(*limits, 100)
y = norm.pdf(x, loc=mean_y_ana, scale=sigma_y_ana)
plt.plot(x, y, label="Analytical")

ax.legend()

La fórmula de propagación de errores es válida si la aproximación lineal de la función $Y = f(X)$ en el entorno alrededor de $X = \mu_X$. En la práctica se puede considerar este entorno como alguna elipse de unos pocos σ ya que contiene la mayoría de la probabilidad de $X$. 